
<h3><font color="#f0c6c6">Generating examples for testing</font></h3>

<font color="#a5adcb"> We have the following prompts, ALL for GPT4 </font>

---


1. <font color="#b7bdf8"> Fewshot (helper.prompt_few_shot) </font> 
2. <font color="#b7bdf8"> Oneshot (helper.prompt_gpt_3) </font> 
3. <font color="#b7bdf8"> Oneshot short (helper.prompt_gpt_short) </font>
4. <font color="#b7bdf8"> Oneshot short extra info (helper.prompt_gpt_short_extra_info) </font>
5. <font color="#b7bdf8"> Oneshot extrainfo + Ingredient rules (helper.prompt_gpt_with_ingredients) </font>

---

<p color="#a5adcb""><font color="#a5adcb"> Few shot examples are bad examples generated by removing rules from good examples, the answer is the original recipe

Extra info signifies that the rules don't have to be in order.

Short prompt is the previous prompt shortened. Hoping to test if longer or shorter prompts generate better examples.
</font></p>

In [1]:
import pandas as pd
import helper 
from gensim.parsing.preprocessing import preprocess_string

DATA_DIR = '../dataset'

extracted_rules, extracted_ingredient_rules = helper.load_required_data_v2(DATA_DIR)

# sample random 100 recipes from the dataset under dataset/full_dataset.csv
seed = 1968
sample_recipes = pd.read_csv(f'{DATA_DIR}/full_dataset.csv').sample(100, random_state=seed)
sample_recipes['directions'] = sample_recipes['directions'].apply(lambda x: eval(x))
# drop recipes that have directions with less than 125 characters in total
sample_recipes['directions_length'] = sample_recipes['directions'].apply(lambda x: len(' '.join(x)))
sample_recipes = sample_recipes[sample_recipes['directions_length'] > 125]

sample_recipes['preprocessed'] = sample_recipes['directions'].apply(lambda x: preprocess_string(' '.join(x)))

Starting to load rule data
Rule data loaded...

Starting rule extraction...
	 -> Starting to sort rules by lift
	 -> Done sorting rules...
______________________________


In [2]:
# grab 8 examples from the middle of the dataset
recipes_to_test = sample_recipes.iloc[45:53]

In [3]:
# grab the first recipe
recipe = recipes_to_test.iloc[0]

fulfilled, suggestions = helper.extract_rules(
    recipe['preprocessed'],
    extracted_rules,
)

In [4]:
fulfilled_ingredients, suggestions_ingredients = helper.extract_rules(
    eval(recipe['NER']),
    extracted_ingredient_rules,
)

In [18]:
from gensim.parsing.preprocessing import preprocess_string
# import stemming from gensim
from gensim.parsing.preprocessing import stem_text

In [20]:
eval(recipe['NER'])

['pastry',
 'golden syrup',
 'heavy cream',
 'bread crumbs',
 'lemon',
 'lemon juice']

In [19]:
for i in eval(recipe['NER']):
    print(stem_text(i))

pastri
golden syrup
heavi cream
bread crumb
lemon
lemon juic


In [5]:
prompt = helper.create_prompt_with_ingredients(
    recipe['title'],
    recipe['directions'],
    fulfilled,
    suggestions,
    fulfilled_ingredients,
    suggestions_ingredients,
)

In [6]:
resp = helper.prompt_gpt_with_ingredients(
    prompt=prompt,
    print_response=False,
    model="gpt-4"
)

In [7]:
recipe['directions']

['Preheat oven to 375 degrees F (190 degrees C). Press the pie crust pastry evenly into the bottom of a 9-inch pie plate.',
 'Mix the syrup, cream, bread crumbs, lemon zest, and lemon juice together in a bowl; pour into the pie crust.',
 'Bake in the preheated oven until the filling is set, 35 to 40 minutes. Allow to cool slightly before serving.']

In [9]:
helper._print_response(helper.GptResponse(resp))

New recipe:

Ingredients:
- 1 pie crust pastry
- 1 cup of syrup
- 1 cup of cream
- 1 cup of bread crumbs
- Zest of 1 lemon
- Juice of 1 lemon
- 1 tablespoon of olive oil

Instructions:
1. Preheat oven to 375 degrees F (190 degrees C). Press the pie crust pastry evenly into the bottom of a 9-inch pie plate. Set aside.
2. In a bowl, mix the syrup, cream, bread crumbs, lemon zest, and lemon juice together. Pour this mixture into the pie crust.
3. Drizzle the olive oil over the mixture in the pie crust.
4. Bake in the preheated oven on the middle rack until the filling is set, 35 to 40 minutes. 
5. Allow to cool slightly before serving. 



________
Explanation:

I added "olive oil" to the ingredients list to fulfill the rule "frozenset({'olive oil'})". I also included it in the instructions by adding a step to drizzle it over the pie mixture before baking. 

To fulfill the rules "frozenset({'asid', 'lemon'})", "frozenset({'asid', 'juic'})", and "frozenset({'rack', 'asid'})", I added "Set 

In [11]:
suggestions_ingredients

{frozenset({'lemon juice'}): ("frozenset({'olive oil'})", 1.8830083502393544)}

In [10]:
suggestions

{frozenset({'juic', 'set'}): ("frozenset({'asid', 'lemon'})",
  23.729338327599788),
 frozenset({'lemon', 'set'}): ("frozenset({'asid', 'juic'})",
  23.690385267142755),
 frozenset({'cool', 'oven', 'set'}): ("frozenset({'rack', 'asid'})",
  18.523671354204385)}